In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class AutoEncoder(nn.Module):
    def __init__(self):

        super().__init__()
        self.encoder = nn.Sequential( # input : 3601*3601
            nn.Conv2d(1, 32, kernel_size=(15, 15), stride=4, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(32, 64, kernel_size=(8, 8), stride=4, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(128, 256, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(256, 512, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(512, 1024, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv2d(1024, 2048, kernel_size=(4, 4), stride=2, padding=1),
            nn.BatchNorm2d(2048),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Flatten()
        )

        self.decoder = nn.Sequential(

            nn.Unflatten(1, (2048, 7, 7)),
            nn.ConvTranspose2d(2048, 1024, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=2, padding=1, output_padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(64, 32, kernel_size=(8, 8), stride=4, padding=1, output_padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.ConvTranspose2d(32, 1, kernel_size=(15, 15), stride=4, padding=1, output_padding=0),
            nn.Sigmoid()

        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    def dimensions(self, x):
        print(f"----------ENCODER----------")
        print(f"Input shape : {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[0](x)
        print(f"After 1st Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[1](x)
        print(f"After 1st ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[2](x)
        print(f"After 2nd Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[3](x)
        print(f"After 2nd ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[4](x)
        print(f"After 3rd Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[5](x)
        print(f"After 3rd ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[6](x)
        print(f"After 4th Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[7](x)
        print(f"After 4th ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[8](x)
        print(f"After 5th Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[9](x)
        print(f"After 5th ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[10](x)
        print(f"After 6th Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[11](x)
        print(f"After 6th ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[12](x)
        print(f"After 7th Conv2d: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[13](x)
        print(f"After 7th ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.encoder[14](x)
        print(f"After Flatten: {x.shape}")


        print("---------DECODER------------")
        x = self.decoder[0](x)
        print(f"After 1st UnFlattend: {x.shape[1], x.shape[2], x.shape[3]}")
        x= self.decoder[1](x)
        print(f"After 1st Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[2](x)
        print(f"After 1st ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[3](x)
        print(f"After 2nd Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[4](x)
        print(f"After 2nd ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[5](x)
        print(f"After 3rd Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[6](x)
        print(f"After 3rd ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[7](x)
        print(f"After 4th Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[8](x)
        print(f"After 4th ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[9](x)
        print(f"After 5th Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[10](x)
        print(f"After 5th ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[11](x)
        print(f"After 6th Transpose: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[12](x)
        print(f"After 6th ReLU: {x.shape[1], x.shape[2], x.shape[3]}")
        x = self.decoder[13](x)
        print(f"After 7th Transpose: {x.shape[1], x.shape[2], x.shape[3]}")



In [4]:
load = torch.load(r'/content/drive/My Drive/Data/DEM_elevation_tensor.pt')

In [5]:
for key in load:
    matrix_min = load[key].min()
    matrix_max = load[key].max()
    load[key] = (load[key] - matrix_min) / (matrix_max - matrix_min)

In [6]:
class DEM_Dataset(Dataset):
    def __init__(self, data):
        self.filename = list(data.keys())
        self.data = data

    def __len__(self):
        return len(self.filename)

    def __getitem__(self, index):
        filename = self.filename[index]
        data = self.data[filename]
        return data

In [7]:
#model = AutoEncoder()

In [8]:
#dummy_input = torch.randn(4, 1, 3601, 3601)
#model.dimensions(dummy_input)

In [9]:
data = DEM_Dataset(load)
batch_size = 4
train_size = 35
test_size = 7
val_size = 8

train_dataset, test_dataset, val_dataset = random_split(data, [train_size, test_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [10]:
model = AutoEncoder()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(15, 15), stride=(4, 4), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv2d(32, 64, kernel_size=(8, 8), stride=(4, 4), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.2, inplace=False)
    (12): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.2, inplace=False)
    (16): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 

In [11]:
from copy import deepcopy

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-5)
num_epochs = 100
best_model = None
best_loss = float('inf')
patience = 10
patience_counter = 0


for epoch in tqdm(range(num_epochs), desc='Epochs'):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        batch_data = batch_data.float().to(device)
        optimizer.zero_grad()
        #print(batch_data.unsqueeze(1).shape)
        #model.dimensions(batch_data.unsqueeze(1))
        outputs = model(batch_data.unsqueeze(1))
        loss = criterion(outputs, batch_data.unsqueeze(1))
        loss.backward()
        optimizer.step()
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for val_batch_data in val_loader:
            val_batch_data = val_batch_data.float().to(device)
            val_outputs = model(val_batch_data.unsqueeze(1))
            val_loss += criterion(val_outputs, val_batch_data.unsqueeze(1)).item()

    val_loss /= len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss}")

     # Check if the validation loss has improved
    if val_loss < best_loss:
        best_loss = val_loss
        best_model = deepcopy(model)  # Save a copy of the model
        patience_counter = 0  # Reset patience counter
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print(f"Early stopping triggered. Stopped at epoch {epoch+1}. Best epoch was {epoch+1-patience_counter} with val loss: {best_loss}")
        break

Epochs:   1%|          | 1/100 [00:10<18:03, 10.95s/it]

Epoch 1/100, Loss: 0.07624200731515884, Val Loss: 0.06146065518260002


Epochs:   2%|▏         | 2/100 [00:13<09:25,  5.77s/it]

Epoch 2/100, Loss: 0.04683155193924904, Val Loss: 0.06134101375937462


Epochs:   3%|▎         | 3/100 [00:15<06:38,  4.11s/it]

...

Epoch 98/100, Loss: 0.054411958903074265, Val Loss: 0.005845693172886968


Epochs:  99%|█████████▉| 99/100 [03:42<00:02,  2.15s/it]

Epoch 99/100, Loss: 0.017578698694705963, Val Loss: 0.0054820633959025145


Epochs: 100%|██████████| 100/100 [03:44<00:00,  2.24s/it]

Epoch 100/100, Loss: 0.014914817176759243, Val Loss: 0.005254655610769987


In [12]:
model.eval()  # Set the model to evaluation mode
test_loss = 0.0
with torch.no_grad():
    with tqdm(test_loader, desc='Testing', leave=False) as t:
        for test_batch_data in t:
            test_batch_data = test_batch_data.float().to(device)
            test_outputs = model(test_batch_data.unsqueeze(1))
            test_loss += criterion(test_outputs, test_batch_data.unsqueeze(1)).item()

test_loss /= len(test_loader)  # Average the test loss
print(f"Test Loss: {test_loss}")

Test Loss: 0.005101608345285058


In [14]:
#torch.save(embedded_dict, "/home/smitesh22/Unsupervised-Machine-Learning-for-Solar-Site-Selection/Code/embedded_dict.pt")

torch.save(model, "/content/drive/My Drive/Thesis/Model/EncoderDecoderElevationModel.pt")